## 1. Install and Import the Required Libraries

In [1]:
!pip install -U -qq langchain==0.3 langchain-openai==0.3 langchain_community==0.3 langchain-experimental==0.3 chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 96.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/1

In [2]:
import os
import sys
from typing import List, Dict, Any, Generator, Optional
import pandas as pd
from datetime import datetime
from email import message_from_string
from email.parser import Parser
from email.policy import default

# LangChain imports
import chromadb
from chromadb.config import Settings
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent

import sqlite3
import re
import warnings
import gc 

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [3]:
# API Key Placeholder: Replace 'YOUR_API_KEY_HERE' with your actual OpenAI API key.
# The script relies on this key for the OpenAIEmbeddingFunction.
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
OPENAI_API_KEY = user_secrets.get_secret("OPENAI_API_KEY")

# use in google colab
#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "YOUR_API_KEY_HERE")

In [4]:
# --- GLOBAL CONFIGURATION (Simulating OpenAI RAG Stack) ---
# Embedding Model (Bi-Encoder): OpenAI's model for quality semantic retrieval
EMBEDDING_MODEL_NAME = 'text-embedding-ada-002' 
# Cross-Encoder (Re-ranker): HuggingFace model for post-retrieval ranking
RE_RANKER_MODEL_NAME = 'cross-encoder/ms-marco-MiniLM-L-6-v2' 
# LLM (Generator): OpenAI's powerful model for synthesis
LLM_MODEL_NAME = 'gpt-3.5-turbo' 

PERSIST_DIRECTORY = "./email_chroma_db"
CHROMA_COLLECTION_NAME = "email_embeddings"

In [5]:
# CSV data file path
file_path = "/kaggle/input/enron-email-dataset/emails.csv"

## Layer 1: The Embedding Layer (Processing, Cleaning, Chunking)

#### Loading and Parsing Email Data

In [6]:
def get_total_count():
    """Get total number of emails without loading entire file"""
    try:
        # Read only first column to count rows efficiently
        df = pd.read_csv(file_path, usecols=[0])
        total_emails = len(df)
        del df
        gc.collect()
    except Exception as e:
        print(f"Warning: Could not get total count: {e}")
        total_emails = 0
        
    return total_emails

total_emails = get_total_count()
print(f'Total emails - {total_emails}')

Total emails - 517401


In [7]:
def parse_email_message(email_text: str) -> Optional[Dict[str, Any]]:
        """
        Parse individual email message using email library
        
        Args:
            email_text: Raw email text
            
        Returns:
            Dictionary containing parsed email fields or None if parsing fails
        """
        try:
            msg = message_from_string(email_text)
            
            # Extract metadata
            email_data = {
                'message_id': msg.get('Message-ID', ''),
                'date': msg.get('Date', ''),
                'from': msg.get('From', ''),
                'to': msg.get('To', ''),
                'subject': msg.get('Subject', ''),
                'cc': msg.get('X-cc', ''),
                'bcc': msg.get('X-bcc', ''),
                'folder': msg.get('X-Folder', ''),
                'origin': msg.get('X-Origin', ''),
            }
            
            # Extract body content
            if msg.is_multipart():
                body = ''
                for part in msg.walk():
                    if part.get_content_type() == 'text/plain':
                        payload = part.get_payload(decode=True)
                        if payload:
                            body += payload.decode('utf-8', errors='ignore')
            else:
                body = msg.get_payload(decode=True)
                if body:
                    body = body.decode('utf-8', errors='ignore')
                else:
                    body = ''
            
            email_data['body'] = body.strip()
            
            # Parse date to datetime object
            try:
                date_str = email_data['date'].split('(')[0].strip()
                email_data['datetime'] = datetime.strptime(
                    date_str,
                    '%a, %d %b %Y %H:%M:%S %z'
                )
            except:
                email_data['datetime'] = None
            
            return email_data
            
        except Exception as e:
            print(f"Error parsing email: {e}")
            return None

In [8]:
def load_emails_chunked() -> Generator[List[Dict[str, Any]], None, None]:
        """
        Load and parse emails in chunks as generator
        
        Yields:
            List of parsed email dictionaries for each chunk
        """
        try:
            # Read CSV in chunks
            chunk_iterator = pd.read_csv(
                file_path,
                chunksize=10000
            )
            
            chunk_num = 0
            total_processed = 0
            
            for chunk_df in chunk_iterator:
                chunk_num += 1
                parsed_chunk = []
                
                for idx, row in chunk_df.iterrows():
                    email_data = parse_email_message(row['message'])
                    if email_data:
                        email_data['file_ref'] = row['file']
                        email_data['row_index'] = idx
                        email_data['chunk_number'] = chunk_num
                        parsed_chunk.append(email_data)
                
                total_processed += len(parsed_chunk)
                
                print(f"Processed chunk {chunk_num}: {len(parsed_chunk)} emails "
                      f"({total_processed}/{total_emails} total)")
                
                yield parsed_chunk
                
                # Force garbage collection after each chunk
                gc.collect()
                
        except Exception as e:
            print(f"Error loading emails in chunks: {e}")
            return

In [9]:
def load_emails_batch(max_emails: Optional[int] = None) -> List[Dict[str, Any]]:
        """
        Load emails in batches but return as single list
        Useful when you need all emails but want memory-efficient loading
        
        Args:
            max_emails: Maximum number of emails to load (None = all)
            
        Returns:
            List of all parsed email dictionaries
        """
        all_emails = []
        emails_loaded = 0
        
        for chunk in load_emails_chunked():
            all_emails.extend(chunk)
            emails_loaded += len(chunk)
            
            if max_emails and emails_loaded >= max_emails:
                all_emails = all_emails[:max_emails]
                break
        
        print(f"Total emails loaded: {len(all_emails)}")
        return all_emails

all_emails = load_emails_batch(total_emails)
email_df = pd.DataFrame(all_emails)

Processed chunk 1: 10000 emails (10000/517401 total)
Processed chunk 2: 10000 emails (20000/517401 total)
Processed chunk 3: 10000 emails (30000/517401 total)
Processed chunk 4: 10000 emails (40000/517401 total)
Processed chunk 5: 10000 emails (50000/517401 total)
Processed chunk 6: 10000 emails (60000/517401 total)
Processed chunk 7: 10000 emails (70000/517401 total)
Processed chunk 8: 10000 emails (80000/517401 total)
Processed chunk 9: 10000 emails (90000/517401 total)
Processed chunk 10: 10000 emails (100000/517401 total)
Processed chunk 11: 10000 emails (110000/517401 total)
Processed chunk 12: 10000 emails (120000/517401 total)
Processed chunk 13: 10000 emails (130000/517401 total)
Processed chunk 14: 10000 emails (140000/517401 total)
Processed chunk 15: 10000 emails (150000/517401 total)
Processed chunk 16: 10000 emails (160000/517401 total)
Processed chunk 17: 10000 emails (170000/517401 total)
Processed chunk 18: 10000 emails (180000/517401 total)
Processed chunk 19: 10000 em

In [10]:
email_df.head(5)

,message_id,date,from,to,subject,cc,bcc,folder,origin,body,datetime,file_ref,row_index,chunk_number
0,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail",Allen-P,Here is our forecast,2001-05-14 16:39:00-07:00,allen-p/_sent_mail/1.,0,1
1,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail",Allen-P,"Traveling to have a business meeting takes the fun out of the trip. Especially if you have to prepare a presentation. I would suggest holding the business plan meetings here then take a trip without any formal business meetings. I would even try and get some honest opinions on whether a trip is even desired or necessary.\n\nAs far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not. Too often the presenter speaks and the others are quiet just waiting for their turn. The meetings might be better if held in a round table discussion format. \n\nMy suggestion for where to go is Austin. Play golf and rent a ski boat and jet ski's. Flying somewhere takes too much time.",2001-05-04 13:51:00-07:00,allen-p/_sent_mail/10.,1,1
2,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,test successful. way to go!!!,2000-10-18 03:00:00-07:00,allen-p/_sent_mail/100.,2,1
3,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,"Randy,\n\n Can you send me a schedule of the salary and level of everyone in the \nscheduling group. Plus your thoughts on any changes that need to be made. \n(Patti S for example)\n\nPhillip",2000-10-23 06:13:00-07:00,allen-p/_sent_mail/1000.,3,1
4,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,Let's shoot for Tuesday at 11:45.,2000-08-31 05:07:00-07:00,allen-p/_sent_mail/1001.,4,1


In [11]:
def get_email_text(email: Dict[str, Any]) -> str:
    """
    Create full-text representation of email for embedding
    
    Args:
        email: Parsed email dictionary
        
    Returns:
        Formatted email text
    """
    text_parts = [
        f"From: {email.get('from', 'Unknown')}",
        f"To: {email.get('to', 'Unknown')}",
        f"Date: {email.get('date', 'Unknown')}",
        f"Subject: {email.get('subject', 'No Subject')}",
    ]
    
    if email.get('cc'):
        text_parts.append(f"CC: {email['cc']}")
    
    text_parts.append(f"\n{email.get('body', '')}")
    
    return "\n".join(text_parts)

#### Email Data Chunking 

In [12]:
import tiktoken

encoding = tiktoken.encoding_for_model(LLM_MODEL_NAME)

def token_length(text: str) -> int:
    """
    Count tokens in text
    
    Args:
        text: Input text
        
    Returns:
        Number of tokens
    """
    tokens = encoding.encode(text)
    return len(tokens)

In [13]:
# Configure text splitter for chunking long emails
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,          # Max chars per chunk
    chunk_overlap=200,        # Overlap to maintain context
    length_function=token_length,
    separators=["\n\n", "\n", ". ", " ", ""]  # Split on paragraphs first
)

In [14]:
def chunk_email(email_text: str, metadata: Dict[str, Any]) -> List[Dict[str, Any]]:
    """
    Chunk a single email into smaller pieces
    
    Args:
        email_text: Full email text
        metadata: Email metadata to attach to each chunk
        
    Returns:
        List of chunk dictionaries with text and metadata
    """
    chunks = text_splitter.split_text(email_text)
    
    chunk_dicts = []
    for idx, chunk in enumerate(chunks):
        chunk_dict = {
            'text': chunk,
            'chunk_index': idx,
            'total_chunks': len(chunks),
            **metadata
        }
        chunk_dicts.append(chunk_dict)
    
    return chunk_dicts

In [15]:
def chunk_emails_streaming(email_chunks: Generator[List[Dict[str, Any]], None, None],
                           get_text_fn) -> Generator[List[Dict[str, Any]], None, None]:
    """
    Chunk multiple emails in streaming fashion
    Processes email chunks as they come without loading all into memory
    
    Args:
        email_chunks: Generator yielding email chunk lists
        get_text_fn: Function to extract text from email dict
        
    Yields:
        Lists of text chunks with metadata
    """
    chunk_count = 0
    
    for email_batch in email_chunks:
        batch_chunks = []
        
        for email in email_batch:
            email_text = get_text_fn(email)
            
            # Create metadata for chunks
            metadata = {
                'message_id': email.get('message_id', ''),
                'from': email.get('from', ''),
                'to': email.get('to', ''),
                    'subject': email.get('subject', ''),
                    'date': email.get('date', ''),
                    'file_ref': email.get('file_ref', ''),
                    'row_index': email.get('row_index', -1),
                    'chunk_number': email.get('chunk_number', 0)
                }
                
            # Chunk the email
            chunks = chunk_email(email_text, metadata)
            batch_chunks.extend(chunks)
            chunk_count += len(chunks)
            
        print(f"Generated {len(batch_chunks)} text chunks (total: {chunk_count})")
        
        yield batch_chunks
        
        # Force garbage collection
        gc.collect()

## Layer 2: The Search Layer (Retrieval, Re-ranking, & Caching)

#### Manage Embeddings and Semanic Search using ChromaDB

In [16]:
# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    model=EMBEDDING_MODEL_NAME
)

In [17]:
chromadb.api.client.SharedSystemClient.clear_system_cache()

In [18]:
# Initialize ChromaDB client
client = chromadb.PersistentClient(
    path=PERSIST_DIRECTORY,
    settings=Settings(anonymized_telemetry=False)
)

In [19]:
def create_vectorstore_streaming(chunk_generator: Generator[List[Dict[str, Any]], None, None]):
        """
        Create vector store from streaming chunks
        More memory-efficient for large datasets
        
        Args:
            chunk_generator: Generator yielding chunk batches
        """
        print(f"Creating vector store with streaming ingestion...")
        
        total_chunks = 0
        first_batch = True
        
        for chunk_batch in chunk_generator:
            # Extract texts and metadata
            texts = [chunk['text'] for chunk in chunk_batch]
            metadatas = [
                {
                    'message_id': chunk.get('message_id', ''),
                    'from': chunk.get('from', ''),
                    'to': chunk.get('to', ''),
                    'subject': chunk.get('subject', ''),
                    'date': chunk.get('date', ''),
                    'chunk_index': str(chunk.get('chunk_index', 0)),
                    'total_chunks': str(chunk.get('total_chunks', 1)),
                    'file_ref': chunk.get('file_ref', ''),
                    'row_index': str(chunk.get('row_index', -1))
                }
                for chunk in chunk_batch
            ]
            
            # Create or add to vector store
            if first_batch:
                # Create initial vectorstore
                vector_store = Chroma.from_texts(
                    texts=texts,
                    embedding=embeddings,
                    metadatas=metadatas,
                    collection_name=CHROMA_COLLECTION_NAME,
                    client=client
                )
                first_batch = False
            else:
                # Add to existing vectorstore
                vector_store.add_texts(
                    texts=texts,
                    metadatas=metadatas
                )
            
            total_chunks += len(texts)
            print(f"Indexed {len(texts)} chunks (total: {total_chunks})")
            
            # Force garbage collection
            gc.collect()
        
        print(f"Vector store created successfully with {total_chunks} chunks!")
        return vector_store

In [20]:
def load_vectorstore():
    """
    Load existing vector store from disk.
    
    Returns:
        Chroma vectorstore instance
        
    Raises:
        FileNotFoundError: If vector store doesn't exist
    """
    if os.path.exists(PERSIST_DIRECTORY):
        print(f"Loading existing vector store from {PERSIST_DIRECTORY}...")
        vector_store = Chroma(
            client=client,
            collection_name=CHROMA_COLLECTION_NAME,
            embedding_function=embeddings
        )
        print("✓ Vector store loaded successfully")
        return vector_store
    else:
        print(f"Vector store not found at {PERSIST_DIRECTORY}. Please build the vector store first.")
        return None

In [21]:
def similarity_search(query: str, k: int = 5) -> List[Dict[str, Any]]:
    """
    Perform similarity search
    
    Args:
        query: Search query
        k: Number of results to return
        
    Returns:
        List of similar documents with metadata
    """
    if vector_store is None:
        raise ValueError(
            "Vector store not initialized. "
            "Please build or load the vector store first."
        )
    
    results = vector_store.similarity_search_with_score(query, k=k)
    
    formatted_results = []
    for doc, score in results:
        formatted_results.append({
            'content': doc.page_content,
            'metadata': doc.metadata,
            'similarity_score': score
        })
    
    return formatted_results

In [22]:
# Check if we need to index emails
if not os.path.exists(PERSIST_DIRECTORY):
    print("\n[STREAMING MODE] Indexing emails ...")
    
    print(f"\n Email Parser Configuration:")
    print(f"  Total Emails: {total_emails}")
    print(f"  Chunk Size: {10000} emails per chunk")
    
    # Stream emails -> chunks -> embeddings
    print("\n[PHASE 1] Parsing emails in chunks...")
    email_chunks_gen = load_emails_chunked()
    
    
    print(f"\n Email Data Chunking Configuration:")
    print(f"  Text Chunk Size: {1000} tokens")
    print(f"  Overlap: {200} tokens")
    
    print("\n[PHASE 2] Generating text chunks...")
    text_chunks_gen = chunk_emails_streaming(
        email_chunks_gen,
        get_email_text
    )
    
    print("\n[PHASE 3] Creating embeddings and indexing...")
    vector_store = create_vectorstore_streaming(text_chunks_gen)
    
    # Force garbage collection
    gc.collect()
        
else:
    print("\n[LOADING] Loading existing vector store...")
    vector_store = load_vectorstore()


[LOADING] Loading existing vector store...
Loading existing vector store from ./email_chroma_db...
✓ Vector store loaded successfully


## Layer 3: The Generation Layer (LLM Simulation)

#### LangChain-powered Intelligent Agents for Email Search

In [23]:
#LLM model to use (gpt-3.5-turbo)
llm = ChatOpenAI(
            temperature=0,                    # Deterministic responses
            model=LLM_MODEL_NAME,
            openai_api_key=OPENAI_API_KEY
        )

#### Conversational Agent

In [24]:
# Initialize conversation memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

# Custom prompt for conversational context
qa_prompt = PromptTemplate(
    template="""You are a helpful and friendly email search assistant. You engage in natural conversations while helping users find information in their email archives.

Previous conversation:
{chat_history}

Context from emails:
{context}

Current question: {question}

Provide a conversational, helpful response that:
1. Acknowledges previous conversation context when relevant
2. Directly answers the current question
3. References specific emails naturally (e.g., "In the email from John on May 14th...")
4. Asks clarifying questions if needed
5. Maintains a friendly, professional tone

Answer:""",
            input_variables=["chat_history", "context", "question"]
        )


# Create conversational retrieval chain
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(
        search_kwargs={"k": 5}
    ),
    memory=memory,
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": qa_prompt}
)

In [25]:
def conversation_chat(message: str) -> Dict[str, Any]:
    """
    Have a conversation turn
    
    Args:
        message: User message
        
    Returns:
        Dictionary with response and sources
    """
    
    try:
        result = conversation_chain({"question": message})
        # For ConversationalRetrievalChain, the default output key is "answer"
        # unless you override output_key.
        answer = result.get("answer", result.get("result", ""))
        response = {
            "status": "success",
            "message": message,
            "response": answer,
            "raw": result,
            "source_documents": [],
        }
        for doc in result.get("source_documents", []):
            source = {
                "content": doc.page_content[:300] + "...",
                "from": doc.metadata.get("from", "Unknown"),
                "to": doc.metadata.get("to", "Unknown"),
                "subject": doc.metadata.get("subject", "No Subject"),
                "date": doc.metadata.get("date", "Unknown"),
            }
            response["source_documents"].append(source)
        return response

    except Exception as e:
        # Always return "response" key so callers don't KeyError
        return {
            "status": "error",
            "message": message,
            "response": f"Error in conversational agent: {e}",
            "raw": None,
            "source_documents": [],
        }

In [26]:
def get_conversation_history() -> List[Dict[str, str]]:
    """
    Get conversation history
    
    Returns:
        List of conversation turns
    """
    history = []
    if hasattr(memory, 'chat_memory'):
        for message in memory.chat_memory.messages:
            history.append({
                'role': message.type,
                'content': message.content
            })
    return history

#### Data Analytics Agent

In [27]:
 def create_analytics_dataframe() -> pd.DataFrame:
    """
    Create structured DataFrame from raw email data
    
    Returns:
        Processed DataFrame with email fields
    """
    print("Creating analytics DataFrame...")
    
    df = email_df
    
    # Feature engineering
    df['body_length'] = df['body'].str.len()
    df['subject_length'] = df['subject'].str.len()
    df['has_subject'] = df['subject'].str.len() > 0
    df['has_cc'] = df['cc'].str.len() > 0
    
    # Extract sender domain
    df['sender_domain'] = df['from'].str.extract(r'@([^\s>]+)')
    
    # Extract date components (simplified)
    df['date_str'] = df['date'].astype(str)
    
    print(f"Analytics DataFrame created with {len(df)} emails and {len(df.columns)} features")
        
    return df

In [28]:
# Create pandas agent
df = create_analytics_dataframe()
analytics_agent = create_pandas_dataframe_agent(
    llm,
    df,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    allow_dangerous_code=True,  # Required for pandas agent
    handle_parsing_errors=True
)

Creating analytics DataFrame...
Analytics DataFrame created with 517401 emails and 20 features


In [29]:
def analyze_emails(query: str) -> Dict[str, Any]:
    """
    Run analytics query
    
    Args:
        query: Natural language analytics query
        
    Returns:
        Analysis results
    """
    try:
        result = analytics_agent.run(query)
        
        return {
            'query': query,
            'result': result,
            'status': 'success'
        }
    except Exception as e:
        return {
            'query': query,
            'result': f"Error: {str(e)}",
            'status': 'error'
        }

#### SQL Agent

In [30]:
sql_db_path = "./emails.db"
def create_database():
    """Create SQLite database from email CSV"""
    print(f"Creating SQLite database at {sql_db_path}...")
    
    df = email_df
    
    # Create SQLite database
    conn = sqlite3.connect(sql_db_path)
    
    # Create main emails table
    df.to_sql('emails', conn, if_exists='replace', index=False)
    
    # Create additional analytics tables
    
    # Senders table
    senders_df = df[['from']].drop_duplicates()
    senders_df['sender_id'] = range(1, len(senders_df) + 1)
    senders_df.to_sql('senders', conn, if_exists='replace', index=False)
        
    # Recipients table
    recipients_df = df[['to']].drop_duplicates()
    recipients_df['recipient_id'] = range(1, len(recipients_df) + 1)
    recipients_df.to_sql('recipients', conn, if_exists='replace', index=False)
    
    # Create indexes
    cursor = conn.cursor()
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_from ON emails("from")')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_to ON emails("to")')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_subject ON emails("subject")')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_date ON emails("date")')
    
    conn.commit()
    conn.close()
    
    print(f"Database created with {len(df)} emails")

In [31]:
# Create SQLite database from emails
create_database()

# Initialize LangChain SQL database wrapper
db = SQLDatabase.from_uri(f"sqlite:///{sql_db_path}")

# Create SQL toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Create SQL agent
sql_agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=False,
    handle_parsing_errors=True
)

Creating SQLite database at ./emails.db...
Database created with 517401 emails


In [32]:
def sql_query(question: str) -> Dict[str, Any]:
    """
    Execute SQL query via natural language
    
    Args:
        question: Natural language question
        
    Returns:
        Query results
    """
    try:        
        raw_result = sql_agent.invoke({"input": question})
            
        return {
            'question': question,
            'result': raw_result['output'],
            'intermediate_steps': raw_result.get('intermediate_steps', []),
            'status': 'success'
        }
    except Exception as e:
        return {
            'question': question,
            'result': f"SQL Agent Error: {str(e)}",
            'status': 'error'
        }

In [33]:
import sqlite3
def execute_raw_sql(sql: str) -> pd.DataFrame:
    """
    Execute raw SQL query
    
    Args:
        sql: SQL query string
        
    Returns:
        Query results as DataFrame
    """
    conn = sqlite3.connect(sql_db_path)
    df = pd.read_sql_query(sql, conn)
    conn.close()
    return df

### Multi Agent Orchestrator

In [34]:
# Types of available agents
agents = {
        "CONVERSATIONAL": "conversation_chain",
        "ANALYTICS": "analytics_agent",
        "SQL": "sql_agent"
    }

In [35]:
def route_query(query: str, agent_type: str) -> Dict[str, Any]:
        """
        Route query to appropriate agent
        
        Args:
            query: User query
            agent_type: Specific agent to use
            
        Returns:
            Agent response
        """
        # Use specified agent 
        target_agent_type = agent_type
        agent = agents.get(target_agent_type)
        
        if not agent:
            return {
                'error': f'Agent {target_agent_type} not available',
                'status': 'error'
            }
        
        # Route to appropriate agent
        try:
            if target_agent_type == "CONVERSATIONAL":
                return conversation_chat(query)
            elif target_agent_type == "ANALYTICS":
                return analyze_emails(query)
            elif target_agent_type == "SQL":
                return execute_raw_sql(query)
        except Exception as e:
            return {
                'error': str(e),
                'status': 'error'
            }

In [36]:
def get_agent_info() -> Dict[str, Any]:
    """
    Get information about available agents
    
    Returns:
        Agent availability and capabilities
    """
    return {
        'available_agents': [
            agent_type
            for agent_type, agent in agents.items() 
            if agent is not None
        ],
        'capabilities': {
            'conversational': 'Natural conversation with memory',
            'analytics': 'Data analysis with pandas',
            'sql': 'SQL queries and structured data retrieval'
        }
    }

In [38]:
# Test 1: Conversational Agent
print("\n" + "─"*80)
print("TEST 1: CONVERSATIONAL AGENT")
print("─"*80)

conv_queries = [
    "Hi! Can you help me search my emails?",
    "I'm looking for emails about business meetings or travel plans.",
    "What about salary discussions?"
]

for query in conv_queries:
    print(f"\nUser: {query}")
    result = route_query(query, "CONVERSATIONAL")
   
    text = result.get("response", result.get("answer", result))
    print(f"Agent: {text}\n")

    if result.get("source_documents"):
        print(f"  [Referenced {len(result['source_documents'])} emails]")


────────────────────────────────────────────────────────────────────────────────
TEST 1: CONVERSATIONAL AGENT
────────────────────────────────────────────────────────────────────────────────

User: Hi! Can you help me search my emails?
Agent: Certainly! To narrow down the search for emails related to salary discussions, could you provide me with any specific details or keywords to help me find the relevant emails? For example, do you remember any specific dates, senders, or topics discussed in those emails? This information will help me locate the emails more efficiently. Feel free to share any details you remember, and I'll do my best to assist you further!


User: I'm looking for emails about business meetings or travel plans.
Agent: Certainly! If you're interested in emails related to salary discussions, I can help you search for those as well. Do you remember any specific details or keywords that could help narrow down the search? For example, were there any particular dates, send

In [39]:
# Test 2: Analytics Agent
print("\n" + "─"*80)
print("TEST 2: DATA ANALYTICS AGENT")
print("─"*80)

analytics_queries = [
    "How many emails are in the dataset?",
    "What is the average email body length?",
    "Show me the top 3 most frequent senders"
]

for query in analytics_queries:
    print(f"\nQuery: {query}")
    result = route_query(query, "ANALYTICS")
    print(f"Result: {result['result']}")


────────────────────────────────────────────────────────────────────────────────
TEST 2: DATA ANALYTICS AGENT
────────────────────────────────────────────────────────────────────────────────

Query: How many emails are in the dataset?


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape[0]'}`


517401There are 517,401 emails in the dataset.

> Finished chain.
Result: There are 517,401 emails in the dataset.

Query: What is the average email body length?


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "average_body_length = df['body_length'].mean()\naverage_body_length"}`


1826.8477911716445The average email body length is approximately 1826.85 characters.

> Finished chain.
Result: The average email body length is approximately 1826.85 characters.

Query: Show me the top 3 most frequent senders


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['from'].value_counts().h

In [40]:
# Test 3: SQL Agent
print("\n" + "─"*80)
print("TEST 3: SQL AGENT")
print("─"*80)

# Test 1: Total email count
print("TOTAL EMAIL COUNT")
print("-" * 40)

sql_query_1 = """
SELECT COUNT(*) AS total_emails
FROM emails;
"""

result1 = route_query(sql_query_1, "SQL")
print(f" SQL:\n{sql_query_1}\n")
print("Total Email Count - SQL Result:")
print(result1)

# Test 2: Top 5 senders by email count
print("\nTOP SENDERS BY EMAIL COUNT")
print("-" * 40)

sql_query_2 = """
SELECT "from", COUNT(*) AS email_count
FROM emails
GROUP BY "from"
ORDER BY email_count DESC
LIMIT 5;
"""

result2 = route_query(sql_query_2, "SQL")
print(f" SQL:\n{sql_query_2}\n")
print("TOP SENDERS BY EMAIL COUNT - SQL Result:")
print(result2)
print()

# Test 3: Emails containing "meeting" in subject (case-insensitive)
print("\nEMAILS ABOUT MEETINGS")
print("-" * 40)

sql_query_3 = """
SELECT file_ref, "from", subject, date
FROM emails
WHERE UPPER(subject) LIKE '%MEETING%'
ORDER BY date DESC
LIMIT 10;
"""

result3 = route_query(sql_query_3, "SQL")
print(f" SQL:\n{sql_query_3}\n")
print("EMAILS ABOUT MEETINGS - SQL Result:")
if not result3.empty:
    for idx, row in result3.iterrows():
        print(f"📧 {row['from']} | {row['subject'][:60]}... | {row['date'][:19]}")
else:
    print("No emails found with 'meeting' in subject")
print()

print("\n" + "="*80)


────────────────────────────────────────────────────────────────────────────────
TEST 3: SQL AGENT
────────────────────────────────────────────────────────────────────────────────
TOTAL EMAIL COUNT
----------------------------------------
 SQL:

SELECT COUNT(*) AS total_emails
FROM emails;


Total Email Count - SQL Result:
   total_emails
0        517401

TOP SENDERS BY EMAIL COUNT
----------------------------------------
 SQL:

SELECT "from", COUNT(*) AS email_count
FROM emails
GROUP BY "from"
ORDER BY email_count DESC
LIMIT 5;


TOP SENDERS BY EMAIL COUNT - SQL Result:
                       from  email_count
0        kay.mann@enron.com        16735
1  vince.kaminski@enron.com        14368
2   jeff.dasovich@enron.com        11411
3      pete.davis@enron.com         9149
4   chris.germany@enron.com         8801


EMAILS ABOUT MEETINGS
----------------------------------------
 SQL:

SELECT file_ref, "from", subject, date
FROM emails
WHERE UPPER(subject) LIKE '%MEETING%'
ORDER BY date 

#### Interactive mode with agent selection

In [43]:
def interactive_multi_agent_mode():
    """
    Interactive mode with agent selection

    """
    print("\n" + "="*80)
    print("INTERACTIVE MULTI-AGENT MODE")
    print("="*80)
    print("\nAvailable Commands:")
    print("  /mode [conv|analytics|sql] - Switch agent mode")
    print("  /info - Show agent information")
    print("  /history - Show conversation history (conversational mode)")    
    print("  /exit - Exit system")
    print("="*80)
    
    agent_info = get_agent_info()    
    print(f"Available Agents: {', '.join(agent_info['available_agents'])}\n")
    current_mode = None
    
    while True:
        try:
            user_input = input("\n> ").strip()
            
            if not user_input:
                continue
            
            # Handle commands
            if user_input.startswith('/'):
                if user_input == '/exit':
                    print("Goodbye!")
                    break
                    
                elif user_input == '/info':
                    info = get_agent_info()
                    print(f"Available Agents: {', '.join(info['available_agents'])}")
                    for agent, capability in info['capabilities'].items():
                        print(f"  • {agent}: {capability}")
                    
                elif user_input.startswith('/mode'):
                    parts = user_input.split()
                    if len(parts) < 2:
                        print("Usage: /mode [conv|analytics|sql]")
                        continue
                    
                    mode_map = {
                        'conv': "CONVERSATIONAL",                        
                        'analytics': "ANALYTICS",
                        'sql': "SQL"
                    }
                    
                    mode = mode_map.get(parts[1])
                    
                    if mode:
                        current_mode = mode
                        print("Current Agent Mode: ", current_mode)
                        print("Current Agent: ", agents[current_mode])
                    else:
                        print("Invalid mode. Use: conv, analytics, or sql")
                
                elif user_input == '/history':
                    if current_mode == "CONVERSATIONAL":
                        history = get_conversation_history()
                        print("\nConversation History:")
                        for turn in history:
                            print(f"{turn['role']}: {turn['content'][:100]}...")
                    else:
                        print("History only available in conversational mode")                                                                      
                else:
                    print("Unknown command. Type /exit to quit.")
                
                continue
            
            # Route query to current agent
            print("\nProcessing...")
            result = route_query(user_input, current_mode)
            
            if result.get('status') == 'error':
                print(f"Error: {result.get('error')}")
                continue
            
            # Display results based on agent type
            if current_mode == "CONVERSATIONAL":
                print(f"User Input:  {user_input}\n")
                text = result.get("response", result.get("answer", result))
                print(f"CONVERSATIONAL Agent: {text}\n")

                if result.get("source_documents"):
                    print(f"  [Referenced {len(result['source_documents'])} emails]")
                    
            elif current_mode == "ANALYTICS":
                print(f"User Input:  {user_input}")
                print(f"\n ANALYTICS Agent: {result['result']}")
                
            elif current_mode == "SQL":
                print(f"User Input:  {user_input}")
                print(f"\n SQL Agent: {result}")
        
        except KeyboardInterrupt:
            print("\n\nExiting...")
            break
        except Exception as e:
            print(f"\nError: {e}")


In [44]:
interactive_multi_agent_mode()


INTERACTIVE MULTI-AGENT MODE

Available Commands:
  /mode [conv|analytics|sql] - Switch agent mode
  /info - Show agent information
  /history - Show conversation history (conversational mode)
  /exit - Exit system
Available Agents: CONVERSATIONAL, ANALYTICS, SQL




>  /info


Available Agents: CONVERSATIONAL, ANALYTICS, SQL
  • conversational: Natural conversation with memory
  • analytics: Data analysis with pandas
  • sql: SQL queries and structured data retrieval



>  /mode analytics


Current Agent Mode:  ANALYTICS
Current Agent:  analytics_agent


...


Processing...
User Input:  provide the email data of meeting happend on Wed, 9 May 2001 19

CONVERSATIONAL Agent: Certainly! I can help you find the email data for the meeting that took place on Wednesday, May 9, 2001, at 7:00 PM. In the email from Sarah on May 10th, there are detailed notes and action items discussed during that meeting. Would you like me to provide you with a summary of the key points or any specific information from that email? Feel free to let me know how I can assist you further!




>  /history



Conversation History:
human: Hi! Can you help me search my emails?...
ai: Of course! I'd be happy to help you search through your emails. Could you please provide me with som...
human: I'm looking for emails about business meetings or travel plans....
ai: Of course! I'd be happy to help you search for emails about business meetings or travel plans. Could...
human: What about salary discussions?...
ai: I see you're looking for emails about business meetings or travel plans. Are you also interested in ...
human: Hi! Can you help me search my emails?...
ai: Certainly! To narrow down the search for emails related to salary discussions, could you provide me ...
human: I'm looking for emails about business meetings or travel plans....
ai: Certainly! If you're interested in emails related to salary discussions, I can help you search for t...
human: What about salary discussions?...
ai: Certainly! I can help you search for emails related to salary discussions as well. Do you recall any...
hum


>  /exit


Goodbye!
